In [3]:
!pip install -q transformers datasets

In [1]:
from datasets import load_dataset
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import re
from tqdm import tqdm
import numpy as np

# Load trivia dataset
dataset = load_dataset("vapit/HarryPotterQA")['train']
validation_dataset = load_dataset("vapit/HarryPotterQA")['validation'].sample(100)

print(f" Train split: {len(dataset)} questions")
print(f" Validation split: {len(validation_dataset)} questions")

# Load GPT Distil
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2889 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [8]:
def normalize(text):
    return re.sub(r"[^a-z0-9]+", " ", text.lower()).strip()

def compute_f1(pred, truth):
    pred_tokens = normalize(pred).split()
    truth_tokens = normalize(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)

In [9]:
def evaluate_model(dataset, n_samples=50):
    exact_match = 0
    f1_scores = []

    for i in tqdm(range(n_samples)):
        item = dataset[i]
        question = item["question"]
        gold = item["answer"]

        prompt = f"Answer this Harry Potter trivia question:\n{question}\nAnswer:"
        output = generator(prompt, max_length=len(tokenizer.encode(prompt)) + 20, do_sample=False)[0]["generated_text"]
        generated = output.split("Answer:")[-1].strip().split("\n")[0]

        norm_pred = normalize(generated)
        norm_gold = normalize(gold)

        print(f"\n📘 Q: {question}")
        print(f"✅ GT: {gold}")
        print(f"🤖 Pred: {generated}")

        if norm_pred == norm_gold:
            exact_match += 1
        f1_scores.append(compute_f1(norm_pred, norm_gold))

    em = exact_match / n_samples
    f1 = np.mean(f1_scores)
    print(f"\n🎯 Exact Match: {em:.2%}")
    print(f"🎯 Average F1: {f1:.2%}")


In [11]:
evaluate_model(validation_dataset, n_samples=50)

  0%|          | 0/50 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=45) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 1/50 [00:01<01:26,  1.76s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=42) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📘 Q: How do the Quidditch team members feel about their new lineup?
✅ GT: They are pleased to be rid of McLaggen and glad to have Katie back, resulting in excellent Quidditch practices under Harry's leadership.
🤖 Pred: I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. I think they're all very happy with the new lineup. 

  4%|▍         | 2/50 [00:03<01:15,  1.56s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📘 Q: How does Sirius Black advise Ron to evaluate Crouch?
✅ GT: Sirius advises Ron to observe Crouch's behavior towards his inferiors to understand his true nature.
🤖 Pred: Ron is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. He is a very good friend of Ron. 

  6%|▌         | 3/50 [00:04<01:10,  1.50s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


📘 Q: What is Mrs. Weasley's motivation for confronting Mundungus Fletcher?
✅ GT: Mrs. Weasley wants to confront Mundungus Fletcher because he failed to follow Harry as instructed. She has also been eager to have a go at him since the incident.
🤖 Pred: Mrs. Weasley is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good friend of Harry Potter. He is a very good frien

 98%|█████████▊| 49/50 [01:13<00:01,  1.56s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📘 Q: Why does the author emphasize a certain aspect?
✅ GT: The author emphasizes their normality to highlight a contrast with the magical events that are about to unfold. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life.
🤖 Pred: I think it's because the author is trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a point that he's trying to make a p

100%|██████████| 50/50 [01:15<00:00,  1.51s/it]


📘 Q: How did Ginny Weasley react to her diary?
✅ GT: Ginny Weasley initially trusted her diary but later became suspicious and tried to dispose of it.
🤖 Pred: I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle of a long night when I was in the middle 

In [23]:
def format_example(example):
    if example["context"].strip():
        return {
            "text": f"Context: {example['context']}\nQuestion: {example['question']}\nAnswer: {example['answer']}"
        }
    else:
        return {
            "text": f"Question: {example['question']}\nAnswer: {example['answer']}"
        }

train_texts = dataset.map(format_example)
val_texts = validation_dataset.map(format_example)

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # required for GPT-style models

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

train_tok = train_texts.map(tokenize, batched=True)
val_tok = val_texts.map(tokenize, batched=True)

Map:   0%|          | 0/2889 [00:00<?, ? examples/s]

In [20]:
!pip install -q transformers datasets accelerate peft

In [25]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],  # GPT2-specific module
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [27]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

args = TrainingArguments(
    output_dir="./peft-gpt2-hp",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to="none",
    fp16=True,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-27-28bc30420065>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
6500,2.932000
13000,2.775300
19500,2.757000


TrainOutput(global_step=19500, training_loss=2.8214248798076924, metrics={'train_runtime': 1257.6863, 'train_samples_per_second': 62.016, 'train_steps_per_second': 15.505, 'total_flos': 5112756392951808.0, 'train_loss': 2.8214248798076924, 'epoch': 3.0})

In [ ]:
def normalize(text):
    return re.sub(r"\W+", " ", text.lower()).strip()

def compute_f1(pred, truth):
    pred_tokens = normalize(pred).split()
    truth_tokens = normalize(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)

In [32]:
from peft import PeftModel
base = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")
model = PeftModel.from_pretrained(base, "./peft-gpt2-hp/checkpoint-19500")

In [33]:
model = PeftModel.from_pretrained(base, "./peft-gpt2-hp/checkpoint-19500")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [34]:
evaluate_model(validation_dataset, n_samples=50)

  0%|          | 0/50 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=45) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 1/50 [00:02<02:04,  2.54s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=25


📘 Q: How do the Quidditch team members feel about their new lineup?
✅ GT: They are pleased to be rid of McLaggen and glad to have Katie back, resulting in excellent Quidditch practices under Harry's leadership.
🤖 Pred: They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous year. They feel that they have a better chance of winning the Quidditch World Cup than they did in the previous ye

  4%|▍         | 2/50 [00:04<01:56,  2.43s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📘 Q: How does Sirius Black advise Ron to evaluate Crouch?
✅ GT: Sirius advises Ron to observe Crouch's behavior towards his inferiors to understand his true nature.
🤖 Pred: Sirius Black advises Ron to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evaluate Crouch by asking him to evalua

  6%|▌         | 3/50 [00:06<01:40,  2.13s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


📘 Q: What is Mrs. Weasley's motivation for confronting Mundungus Fletcher?
✅ GT: Mrs. Weasley wants to confront Mundungus Fletcher because he failed to follow Harry as instructed. She has also been eager to have a go at him since the incident.
🤖 Pred: She is motivated to confront Mundungus Fletcher because she believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good friend. She believes Mundungus Fletcher is a good

 98%|█████████▊| 49/50 [01:31<00:01,  1.78s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📘 Q: Why does the author emphasize a certain aspect?
✅ GT: The author emphasizes their normality to highlight a contrast with the magical events that are about to unfold. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life.
🤖 Pred: The author emphasizes their normality to highlight a contrast with the magical events that are about to unfold. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life. It sets the stage for showing how unusual the upcoming events are compared to the Dursleys' everyday life. It sets the stage for showing how unusual the upcoming events are 

100%|██████████| 50/50 [01:32<00:00,  1.86s/it]


📘 Q: How did Ginny Weasley react to her diary?
✅ GT: Ginny Weasley initially trusted her diary but later became suspicious and tried to dispose of it.
🤖 Pred: Ginny Weasley reacted to her diary by saying: "I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know what she meant by that. I don't know wha

In [38]:
!pip install -q bert-score evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [41]:
# Install required libraries

# Imports
from datasets import load_dataset
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm
import evaluate

# Load dataset
train_dataset = load_dataset("vapit/HarryPotterQA")['train']
validation_dataset = load_dataset("vapit/HarryPotterQA")['validation'].select(range(50))

print(f"Train split: {len(train_dataset)} questions")
print(f"Validation split: {len(validation_dataset)} questions")

# Load model on GPU
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Load evaluation metrics
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("sacrebleu")

# Generate predictions and collect references
predictions = []
references = []

print("Generating responses...")
for sample in tqdm(validation_dataset):
    question = sample['question']
    reference = sample['answer']

    prompt = f"Q: {question}\nA:"
    output = generator(prompt, max_length=100, num_return_sequences=1, do_sample=True, temperature=0.7)[0]['generated_text']

    answer_start = output.find("A:") + 2
    answer = output[answer_start:].strip().split('\n')[0]

    predictions.append(answer)
    references.append(reference)

# Evaluate BLEU (CPU is sufficient)
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print(f"\nBLEU Score: {bleu_score['score']:.2f}")

# Evaluate BERTScore (GPU-accelerated)
print("Computing BERTScore...")
bertscore_result = bertscore.compute(predictions=predictions,
                                     references=references,
                                     lang="en",
                                     device="cuda" if torch.cuda.is_available() else "cpu")

avg_f1 = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
print(f"BERTScore (F1): {avg_f1:.4f}")


Train split: 25999 questions
Validation split: 50 questions


Device set to use cuda:0


Generating responses...


  0%|          | 0/50 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 1/50 [00:01<01:16,  1.57s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation f


BLEU Score: 0.25
Computing BERTScore...


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.8357


In [42]:
model = PeftModel.from_pretrained(base, "./peft-gpt2-hp/checkpoint-19500")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("sacrebleu")

# Generate predictions and collect references
predictions = []
references = []

print("Generating responses...")
for sample in tqdm(validation_dataset):
    question = sample['question']
    reference = sample['answer']

    prompt = f"Q: {question}\nA:"
    output = generator(prompt, max_length=100, num_return_sequences=1, do_sample=True, temperature=0.7)[0]['generated_text']

    answer_start = output.find("A:") + 2
    answer = output[answer_start:].strip().split('\n')[0]

    predictions.append(answer)
    references.append(reference)

# Evaluate BLEU (CPU is sufficient)
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print(f"\nBLEU Score: {bleu_score['score']:.2f}")

# Evaluate BERTScore (GPU-accelerated)
print("Computing BERTScore...")
bertscore_result = bertscore.compute(predictions=predictions,
                                     references=references,
                                     lang="en",
                                     device="cuda" if torch.cuda.is_available() else "cpu")

avg_f1 = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
print(f"BERTScore (F1): {avg_f1:.4f}")


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Device set to use cuda:0


Generating responses...


  0%|          | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 1/50 [00:01<01:30,  1.85s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  4%|▍         | 2/50 [00:04<01:44,  2.17s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for mo


BLEU Score: 5.91
Computing BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.8589


In [44]:

# Imports
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm
import evaluate

# Load dataset
dataset = load_dataset("vapit/HarryPotterQA")
train_dataset = dataset["train"]
validation_dataset = dataset["validation"].select(range(50))  # limit to 50 samples

print(f"Using {len(train_dataset)} training answers as retrieval database.")
print(f"Evaluating on {len(validation_dataset)} validation samples.")

# Load sentence embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Embed training answers (contexts)
train_answers = train_dataset["answer"]
train_embeddings = embedder.encode(train_answers, convert_to_tensor=True, show_progress_bar=True)

# Load generator model
gen_model = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(gen_model)
model = AutoModelForCausalLM.from_pretrained(gen_model).to("cuda")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Evaluate on 50 validation samples
predictions = []
references = []

print("Generating RAG-style answers...")
for sample in tqdm(validation_dataset):
    question = sample["question"]
    reference = sample["answer"]

    # Embed question
    question_embedding = embedder.encode(question, convert_to_tensor=True)

    # Retrieve top-k similar answers from training set
    cosine_scores = util.pytorch_cos_sim(question_embedding, train_embeddings)[0]
    top_k = torch.topk(cosine_scores, k=3)  # Top-3 retrieved answers
    retrieved_contexts = [train_answers[idx] for idx in top_k.indices]

    # Concatenate context + question
    context = "\n".join(retrieved_contexts)
    prompt = f"Context:\n{context}\n\nQ: {question}\nA:"

    # Generate answer
    output = generator(prompt, max_length=150, do_sample=True, temperature=0.7, num_return_sequences=1)[0]['generated_text']
    generated_answer = output.split("A:")[-1].strip().split('\n')[0]

    predictions.append(generated_answer)
    references.append(reference)

# Load evaluation metrics
bleu = evaluate.load("sacrebleu")
bertscore = evaluate.load("bertscore")

# BLEU Score
bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
print(f"\nBLEU Score (manual RAG, 50 samples): {bleu_score['score']:.2f}")

# BERTScore
print("Computing BERTScore...")
bertscore_result = bertscore.compute(predictions=predictions,
                                     references=references,
                                     lang="en",
                                     device="cuda" if torch.cuda.is_available() else "cpu")
avg_f1 = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
print(f"BERTScore (F1, manual RAG): {avg_f1:.4f}")


Using 25999 training answers as retrieval database.
Evaluating on 50 validation samples.


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

Device set to use cuda:0


Generating RAG-style answers...


  0%|          | 0/50 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
  2%|▏         | 1/50 [00:01<01:19,  1.63s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation f


BLEU Score (manual RAG, 50 samples): 0.36
Computing BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1, manual RAG): 0.8091
